In [1]:
import os

TRANSACTION_FILE = "transactions.txt"
USER_FILE = "users.txt"

In [2]:
# Record User to file.
def record_user(account):
    with open(USER_FILE, "a") as file:
        file.write(f"{account['title']},{account['cnic']},{account['phone']},{account['dob']},{account['balance']}\n")

In [3]:
# Update balance in USERS.txt
def update_user_balance(cnic_filter, new_balance):
    if not os.path.exists(USERS_FILE):
        print("User list file not found.")
        return

    with open(USERS_FILE, "r") as file:
        users = file.readlines()

    updated_users = []
    for user in users:
        title, cnic, phone, dob, balance = user.strip().split(",")
        if cnic_filter == cnic:
            updated_users.append(f"{title},{cnic},{phone},{dob},{new_balance:.2f}\n")
        else:
            updated_users.append(user)

    with open(USERS_FILE, "w") as file:
        file.writelines(updated_users)

In [4]:
# Load User with CNIC
def load_user(cnic_filter):
    if not os.path.exists(USERS_FILE):
        print("No user data found.")
        return None

    with open(USERS_FILE, "r") as file:
        for user in users:
            title, cnic, phone, dob, balance = user.strip().split(",")
            if cnic_filter == cnic:
                print(f"Account Title: {title} (CNIC: {cnic}) loaded with balance ${balance}.")
                return {
                    "account_title": name,
                    "cnic": cnic,
                    "phone": phone,
                    "dob": dob,
                    "balance": float(balance),
                    "transactions": []
                }
    print(f"No user found with name {name}.")
    return None

In [23]:
# Record Transaction to file.
# transaction_type: Deposit, Withdrawal
def record_transaction(title, cnic, transaction_type, amount, balance):
    with open(TRANSACTION_FILE, "a") as file:
        file.write(f"{title},{cnic},{transaction_type},{amount},{balance}\n")

In [5]:
# Create an account
def create_account(title, cnic, dob, phone = "",initial_balance=0.0):
    print(f"Account for {name} created with balance ${initial_balance}.")
    account = {
        "account_title": name,
        "cnic": cnic,
        "phone": phone,
        "dob": dob,
        "balance": initial_balance,
        "transactions": []
    }
    record_user(account)
    print(f"Account for {name} created with balance ${initial_balance}.")
    return account

In [6]:
# Deposit function
def deposit(account, amount):
    if amount <= 0:
        print("Error: Deposit amount must be positive.")
        return account
    
    account["balance"] += amount
    account["transactions"].append(("Deposit", amount, account["balance"]))
    record_transaction(account["name"], "Deposit", amount, account["balance"])
    update_user_balance(account["cnic"], account["balance"])
    print(f"{account['name']} deposited ${amount}. New balance: ${account['balance']:.2f}.")
    return account

In [18]:
# Withdraw function
def withdraw(account, amount):
    if amount <= 0:
        print("Error: Withdrawal amount must be positive.")
        return account
    
    if amount > account["balance"]:
        print("Error: Insufficient balance.")
        return account
    
    account["balance"] -= amount
    account["transactions"].append(("Withdrawal", amount, account["balance"]))
    record_transaction(account["name"], "Withdrawal", amount, account["balance"])
    print(f"{account['name']} withdrew ${amount}. New balance: ${account['balance']:.2f}.")
    return account